# VPC Endpoint
- VPC 엔드포인트를 통해 인터넷 게이트웨이, NAT device, VPN 연결 또는 AWS Direct Connect 연결을 필요로 하지 않고 AWS PrivateLink 구동 지원 AWS 서비스 및 VPC 엔드포인트 서비스에 비공개로 연결할 수 있습니다. VPC의 인스턴스는 서비스의 리소스와 통신하는 데 퍼블릭 IP 주소를 필요로 하지 않습니다. VPC와 기타 서비스 간의 트래픽은 Amazon 네트워크를 벗어나지 않습니다.
- 외부 인터넷을 거치지 않고 AWS 서비스에 연결시켜주는 리소스
- Interface Endpoint : ENI(Elastic Network Interface) 기반
    - Private IP를 만들어 서비스로 연결
    - 많은 서비스들을 지원 (SQS, SNS, Kinesis, Sagemaker 등)
    - 서브넷 지정 필요
    - 보안그룹 / 정책을 통해 보호 가능
- Gateway Endpoint : 라우팅 테이블에서 경로의 대상으로 지정하여 사용
    - S3, DynamoDB 지원
    - 정책 적용 가능 (보안그룹은 불가능 : 보안그룹은 ENI 단위로 적용되니까.)
- 상시 비용 발생

## 실습 - VPC Endpoint 활용
    - S3 Gateway Endpoint 만들기
        - Private Instance에서 S3 데이터 조회해보기
    - SQS Interface Endpoint 만들기
        - Private Instance에서 SQS 조회해보기

## 실습 순서
1. 리전은 도쿄
2. VPC 생성 (자동)
    - CIDR : 10.0.0.0/16
    - 가용역역 2개
    - 퍼블릭 서브넷 2개
    - 프라이빗 서브넷 2개
    - NAT GW 없음
    - VPC Endpoint 없음
3. IAM 역할 부여
    - 역할 생성 (EC2) 
    - AmazonS3fullaccess, sqsfull
4. 퍼블릭 인스턴스 생성
    - 키페어 사용
    - VPC, Public Subnet, Public IP 부여
    - 고급 세부 정보 : IAM 인스턴스 프로파일 지정
5. 보안 그룹 수정
    - 인바운드, 아웃바운드 모두 열어주기
6. S3 프로비전
    - 버킷 만들기 : 버킷 이름 뒤에 "-계정아이디" 입력
7. SQS 프로비전
    - 대기열 생성 : 기본으로 두고 생성
8. 인스턴스 연결
    - S3 버킷 리스트 출력 : aws s3 ls
    - SQS 대기열 목록 출력 : aws sqs list-queues
9. 프라이빗 인스턴스 생성
    - 키페어 사용
    - VPC, Private Subnet 선택
    - 기존 보안그룹
    - IAM 프로파일 적용
10. 프라이빗 인스턴스 접속 (배스쳔 호스트)
    - 퍼블릭 EC2에서 keyfile.pem 만들어서 ssh 로 접속
    - aws s3 ls 입력해보면, 경로가 없기 떄문에 반응이 없음
11. VPC Endpoint 생성
    - 게이트웨이 엔드포인트 생성
    - 유형 : AWS 서비스
    - 서비스 : S3 검색해서 Gateway유형 선택
    - 네트워크 설정에서 VPC 선택
    - 라우팅 테이블 업데이트 : 모두 선택
12. 프라이빗 인스턴스에서 연결 확인
    - aws s3 ls
13. SQS Endpoint 생성
    - sqs 서비스 검색해서 Interface 유형 선택
    - 서브넷 선택 (아무거나)
14. 프라이빗 인스턴스에서 연결 확인
    - aws sqs list-queues
15. 리소스 정리
    - VPC 엔드포인트 삭제
    - 인스턴스 모두 종료
    - VPC 삭제